<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 2 - Singapore Housing Data and Kaggle Challenge - PART 4

---
# PART 4 - Test Set Transformations, Kaggle Export


## 4.1 Dealing with Test dataset

Import required libraries:

In [1]:
import pandas as pd
from scipy.stats import ttest_ind
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge

import seaborn               as sns
import matplotlib.pyplot     as plt
import matplotlib.ticker as mticker

import statsmodels.api as sm
import pickle
import joblib

%matplotlib inline

In [2]:
# Setting the print options to display full numbers (no scientific notation)
pd.options.display.float_format = '{:.2f}'.format

Import test data sets:

In [3]:
test = pd.read_csv('./data/test.csv', low_memory=False)

In [4]:
print(test.shape)
test.head()

(16737, 76)


,id,Tranc_YearMonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,vacancy,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude
0,114982,2012-11,YISHUN,4 ROOM,173,YISHUN AVE 7,07 TO 09,84.00,Simplified,1987,...,92,0,1.43,103.83,156.32,Ahmad Ibrahim Secondary School,218,0,1.44,103.83
1,95653,2019-08,JURONG WEST,5 ROOM,986C,JURONG WEST ST 93,04 TO 06,112.00,Premium Apartment,2008,...,45,0,1.34,103.70,739.37,Jurong West Secondary School,199,0,1.34,103.70
2,40303,2013-10,ANG MO KIO,3 ROOM,534,ANG MO KIO AVE 10,07 TO 09,68.00,New Generation,1980,...,36,0,1.37,103.85,305.07,Anderson Secondary School,245,0,1.37,103.85
3,109506,2017-10,WOODLANDS,4 ROOM,29,MARSILING DR,01 TO 03,97.00,New Generation,1979,...,54,0,1.43,103.77,433.45,Woodlands Secondary School,188,0,1.44,103.77
4,100149,2016-08,BUKIT BATOK,4 ROOM,170,BT BATOK WEST AVE 8,16 TO 18,103.00,Model A,1985,...,40,0,1.35,103.74,217.30,Bukit Batok Secondary School,223,0,1.35,103.74


---
### 4.1.1 General data "cleaning"
* Addressing potential nulls by filling with zero
* Datatype conversion
* Binary mapping
* Apply snake case to column names

Since we cannot be sure of what null values and in which columns would they appear, drop all rows with nulls.

In [5]:
test.fillna(0, inplace=True)

In [6]:
#convert tranc yearmonth to datetime dtype
test['Tranc_YearMonth'] = pd.to_datetime(test['Tranc_YearMonth'], format='%Y-%m')

In [7]:
#identify boolean columns which are not yet converted to 1/0s
bool_col_test = test[['residential', 'commercial', 'market_hawker', 'multistorey_carpark', 'precinct_pavilion']]

#map these columns from Y/N to 1/0
for col in bool_col_test.columns:
    test[col] = test[col].map({'Y': 1, 'N': 0})

In [8]:
#make column names snake_case
test.columns = test.columns.str.lower()

---
### 4.1.2 Feature Selection
* Numerical features
* Categorical features
* Merging numerical and categorical dataframe

#### Numerical features

Extract numerical features:

In [9]:
#Extract numerical features
test_num = test.select_dtypes(include=['float64','int64'])

Drop all columns dropped in train set:

In [10]:
#drop columns
test_num.drop(['id', 'residential',
                'lower', 'upper', 'mid', 'latitude', 'longitude', 'mrt_latitude', 'mrt_longitude', 'bus_stop_latitude', 'bus_stop_longitude', 'pri_sch_latitude', 
                'pri_sch_longitude', 'sec_sch_latitude','sec_sch_longitude',
               'id', 'residential', 
               'commercial', 'market_hawker', 'multistorey_carpark', 'precinct_pavilion',
                       'bus_interchange', 'mrt_interchange', 'pri_sch_affiliation', 'affiliation',
               'mall_within_500m', 'mall_within_1km', 'hawker_within_500m', 'hawker_within_1km',
              'multigen_sold', '1room_rental', '3room_rental', 'other_room_rental',
              'floor_area_sqft', 'year_completed', 'lease_commence_date'], axis=1, inplace=True)

In [11]:
test_num.shape

(16737, 27)

---
#### Categorical features

Extract categorical features (Exclude those dropped in trainset)

In [12]:
Cat_feat = ['town', 'street_name', 'full_flat_type', 'planning_area', 'mrt_name',
       'pri_sch_name', 'sec_sch_name']

Bin_feat = ['commercial', 'market_hawker', 'multistorey_carpark',
       'precinct_pavilion', 'bus_interchange', 'mrt_interchange',
       'pri_sch_affiliation', 'affiliation']

In [13]:
columns_cat = Cat_feat + Bin_feat
test_cat = test[columns_cat]

In [14]:
test_cat.shape

(16737, 15)

Addressing planning area with <1000 value count:

In [15]:
planning_area_count = test_cat['planning_area'].value_counts()

#extract planning areas whose counts <1000
planning_area_below_1000 = planning_area_count[planning_area_count<1000]

# Converting planning areas <1000 to be planning_area_others
test_cat['planning_area'] = test_cat['planning_area'].apply(lambda x: 'planning_area_others' if x in planning_area_below_1000 else x)

C:\Users\glori\AppData\Local\Temp\ipykernel_864\2285560018.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_cat['planning_area'] = test_cat['planning_area'].apply(lambda x: 'planning_area_others' if x in planning_area_below_1000 else x)


---
### 4.1.3 Further data transformations
* Scaling to Numerical features
* OHE to Categorical features
* Merge

#### Scaling to numerical features

Import train dataset to get training cat df to fit the standardscaler:

In [16]:
train = pickle.load(open('./pkl/train_sel.pkl', 'rb'))

In [17]:
#define X predictor variables, and y target variable
X = train.drop(columns='resale_price')
y = train['resale_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Cat_feat = ['town', 'street_name', 'full_flat_type', 'planning_area', 'mrt_name',
       'pri_sch_name', 'sec_sch_name']

Bin_feat = ['commercial', 'market_hawker', 'multistorey_carpark',
       'precinct_pavilion', 'bus_interchange', 'mrt_interchange',
       'pri_sch_affiliation', 'affiliation']

# Drop cat values to get X_train_num
columns_cat = Cat_feat + Bin_feat
X_train_num = X_train.drop(columns=columns_cat, inplace=False)

In [18]:
num_columns = test_num.columns.tolist()

In [19]:
ss = StandardScaler()

# Fit the StandardScaler on the training data
X_train_num[num_columns] = ss.fit_transform(X_train_num[num_columns])

# Scale the numerical columns in the test_cat DataFrame using the fitted StandardScaler
test_num[num_columns] = ss.transform(test_num[num_columns])

In [20]:
test_num.head()

,floor_area_sqm,tranc_year,tranc_month,mid_storey,hdb_age,max_floor_lvl,total_dwelling_units,1room_sold,2room_sold,3room_sold,...,hawker_nearest_distance,hawker_within_2km,hawker_food_stalls,hawker_market_stalls,mrt_nearest_distance,bus_stop_nearest_distance,pri_sch_nearest_distance,vacancy,sec_sch_nearest_dist,cutoff_point
0,-0.54,-1.63,1.32,-0.05,0.45,-0.50,0.13,-0.02,-0.15,-0.57,...,-0.45,-0.70,0.38,1.16,-0.18,-0.71,0.13,2.05,-1.14,0.39
1,0.60,0.91,0.42,-0.59,-1.28,-0.18,-1.23,-0.02,-0.15,-0.57,...,0.88,-0.94,1.21,0.64,-1.38,-0.47,0.19,-0.58,0.74,-0.56
2,-1.20,-1.27,1.02,-0.05,1.03,-0.50,1.60,-0.02,-0.15,2.22,...,-0.95,1.80,0.07,0.75,-0.16,-0.52,-0.17,-1.08,-0.66,1.74
3,-0.01,0.18,1.02,-1.14,1.11,-0.18,-0.35,-0.02,-0.15,-0.57,...,-0.62,-0.45,0.17,0.97,0.82,-0.12,2.27,-0.08,-0.24,-1.11
4,0.23,-0.18,0.42,1.58,0.62,1.58,0.33,-0.02,-0.15,-0.57,...,-0.68,-0.45,0.59,0.52,0.52,-0.03,-0.36,-0.86,-0.94,0.64


---
#### OHE to categorical features

In [21]:
columns_cat

['town',
 'street_name',
 'full_flat_type',
 'planning_area',
 'mrt_name',
 'pri_sch_name',
 'sec_sch_name',
 'commercial',
 'market_hawker',
 'multistorey_carpark',
 'precinct_pavilion',
 'bus_interchange',
 'mrt_interchange',
 'pri_sch_affiliation',
 'affiliation']

In [22]:
X_train_cat = train[columns_cat]

In [23]:
X_train_cat = pd.get_dummies(X_train_cat[columns_cat])

In [24]:
# Fit the encoder on the training data using the common columns
test_cat = pd.get_dummies(test_cat[columns_cat])

# Make sure the column names of the test data match the training data
test_cat = test_cat.reindex(columns=X_train_cat.columns, fill_value=0)

In [25]:
test_cat.shape

(16737, 1060)

---
#### Merging numerical and categorical dataframe

In [26]:
#Retrieve final Z_train for use to instantiate model again in this notebook.
# Combine Z_train and X_train_num along axis=1 (column-wise)
Z_train = pd.concat([X_train_num, X_train_cat], axis=1)

In [27]:
test_final = pd.concat([test_num, test_cat], axis=1)

In [28]:
print(test_final.shape)
test_final.head()

(16737, 1087)


,floor_area_sqm,tranc_year,tranc_month,mid_storey,hdb_age,max_floor_lvl,total_dwelling_units,1room_sold,2room_sold,3room_sold,...,sec_sch_name_Xinmin Secondary School,sec_sch_name_Yio Chu Kang Secondary School,sec_sch_name_Yishun Secondary School,sec_sch_name_Yishun Town Secondary School,sec_sch_name_Yuan Ching Secondary School,sec_sch_name_Yuhua Secondary School,sec_sch_name_Yusof Ishak Secondary School,sec_sch_name_Yuying Secondary School,sec_sch_name_Zhenghua Secondary School,sec_sch_name_Zhonghua Secondary School
0,-0.54,-1.63,1.32,-0.05,0.45,-0.50,0.13,-0.02,-0.15,-0.57,...,0,0,0,0,0,0,0,0,0,0
1,0.60,0.91,0.42,-0.59,-1.28,-0.18,-1.23,-0.02,-0.15,-0.57,...,0,0,0,0,0,0,0,0,0,0
2,-1.20,-1.27,1.02,-0.05,1.03,-0.50,1.60,-0.02,-0.15,2.22,...,0,0,0,0,0,0,0,0,0,0
3,-0.01,0.18,1.02,-1.14,1.11,-0.18,-0.35,-0.02,-0.15,-0.57,...,0,0,0,0,0,0,0,0,0,0
4,0.23,-0.18,0.42,1.58,0.62,1.58,0.33,-0.02,-0.15,-0.57,...,0,0,0,0,0,0,0,0,0,0


---
## 4.2 Export Test Final

In [29]:
import os # to work with files/directories
if not os.path.exists('./data/output'): 
    os.makedirs('./data/output') 

# Save the DataFrame to a CSV file
test_final.to_csv('./data/output/test_final.csv', index=False)

In [30]:
#Export pre-pickled dataframe
pickle.dump(test_final, open('./pkl/test_final.pkl', 'wb'))